In [ ]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date, datetime, timedelta
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
import random
from sklearn.model_selection import train_test_split
import tensorflow.keras.backend as K
from scipy.stats import norm
from sklearn.preprocessing import Normalizer,StandardScaler, LabelEncoder
from sklearn.metrics import mean_pinball_loss
from scipy import stats
import math
import optuna as opt
from sklearn.model_selection import KFold
from losses import *

# Tune Wind model

## Functions

In [ ]:
def normalize(dataframe, label_encoder = None,feature_scaler = None, target_scaler = None, learn = False):
    #Drop unused columns
    data = dataframe.copy()
    data.drop(["init_tm", "met_var", "location",  "ens_var", "obs_tm"], axis = 1, inplace = True)
    data = data.to_numpy()
    if learn == True:
        label_encoder = LabelEncoder()
        feature_scaler = StandardScaler()
        target_scaler = StandardScaler()
        #Learn label encoding for horizons
        label = label_encoder.fit_transform(data[:,0])
        #Learn target scaling
        target_scaled = target_scaler.fit_transform(data[:,1].reshape(-1,1))
        #Learn feature scaling
        feature_scaled = feature_scaler.fit_transform(data[:,2:])
        #Append
        data[:,0] = label
        data[:,1] = target_scaled.reshape(-1)
        data[:,2:] = feature_scaled
        
        return data, label_encoder, feature_scaler, target_scaler
    
    else:
        #Learn labels
        label = label_encoder.transform(data[:,0])
        #Scale target
        target_scaled = target_scaler.transform(data[:,1].reshape(-1,1))
        #Scale features
        feature_scaled = feature_scaler.transform(data[:,2:])
        #Append
        data[:,0] = label
        data[:,1] = target_scaled.reshape(-1)
        data[:,2:] = feature_scaled
        
        return data

In [ ]:
def convert_format(input_data, predict = False):
    #Extract forecast embedding
    horizon_emb = input_data[:,0]
    
    if predict == False:        
        #Extract features
        features = input_data[:,2:]
        # Extract target
        target = np.expand_dims(input_data[:,1],1)
        return [features, horizon_emb], target
    else:
        #Extract features
        features = input_data[:,1:]
        return [features, horizon_emb]

In [ ]:
def train_model(train_data, train_target, validation_data, batch_size, epochs, learning_rate, fine_tuning = True):
    model = base_model()    
    #Define optimizer
    optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)
    #Callbacks
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience = 7, min_delta = 1e-5)
    model.compile(optimizer = optimizer, loss = lambda true,pred: pinball_loss(true, pred, tau = quantiles))
    #model.compile(optimizer = optimizer, loss = lambda true,pred: smooth_pinball_loss(true, pred, tau = quantiles))
    #Normal fit
    history1 = model.fit(x = train_data, y = train_target, validation_data = validation_data, epochs = epochs, batch_size = batch_size, callbacks = [callback], shuffle = True, verbose = False)
    
    #Fine tuning
    if fine_tuning == True:
        enc_horizons = label_encoder.transform(horizons)
        train_filtering = np.isin(train_data[1], enc_horizons)
        train_data_fine = [train_data[0][train_filtering], train_data[1][train_filtering]]
        train_target_fine = train_target[train_filtering]
        #Val filtering
        val_data, val_target = validation_data
        val_filtering = np.isin(val_data[1], enc_horizons)
        val_data_fine = [val_data[0][val_filtering], val_data[1][val_filtering]]
        val_target_fine = val_target[val_filtering]
        validation_data_fine = (val_data_fine, val_target_fine)
        
        #New optimizer
        history2 = model.fit(x = train_data_fine, y = train_target_fine, validation_data = validation_data_fine, epochs = epochs, batch_size = 256, callbacks = [callback], shuffle = True, verbose = False)
    return model, [history1, history2]

## Read data and prepare

In [ ]:
quantiles = [0.025, 0.25, 0.5, 0.75, 0.975]
horizons = [36, 48 ,60, 72, 84]
n_encodings = 65

In [ ]:
#Wind data
wind_data = pd.read_feather("data/berlin_data/historic_data/icon_eps_wind_10m.feather")
#Pressure data
pressure_data = pd.read_feather("data/berlin_data/historic_data/icon_eps_mslp.feather")
pressure_data.rename({"ens_mean":"mean_pressure"}, axis = 1, inplace = True)
#Cloud data
cloud_data = pd.read_feather("data/berlin_data/historic_data/icon_eps_clct.feather")
cloud_data.rename({"ens_mean":"cloud_coverage"}, axis = 1, inplace = True)
#Vmax data
max_data = pd.read_feather("data/berlin_data/historic_data/icon_eps_vmax_10m.feather")
max_data.rename({"ens_mean":"vmax"}, axis = 1, inplace = True)


data = wind_data.merge(pressure_data[["init_tm","fcst_hour","mean_pressure"]], on = ["init_tm","fcst_hour"], how = "left")
data = data.merge(cloud_data[["init_tm","fcst_hour","cloud_coverage"]], on = ["init_tm","fcst_hour"], how = "left")
data = data.merge(max_data[["init_tm","fcst_hour","vmax"]], on = ["init_tm","fcst_hour"], how = "left")
#Replace vmax NaNs by mean
vmax_mean = data["vmax"].mean()
data.loc[:,"vmax"].fillna(vmax_mean, inplace = True)
data.dropna(inplace=True)

#Positional encoding
pos_enc = pd.DataFrame(index=pd.DatetimeIndex(data["obs_tm"]))
pos_enc["Dayofyear"] = pos_enc.index.dayofyear
pos_enc["n_days"] = 365
pos_enc.loc[pos_enc.index.year==2020,"n_days"] = 366
#Calculate actual positional encoding
cos_encoding = np.cos(2*math.pi*pos_enc["Dayofyear"]/pos_enc["n_days"])
data["pos_enc_1"] = cos_encoding.to_numpy()

## Create study object

In [ ]:
def get_data(train_index, test_index, data):
    #Get split
    train_df = data.loc[train_index]
    test_df = data.loc[test_index]

    #Normalize data
    train, label_encoder, feature_scaler, target_scaler = normalize(train_df, learn = True)
    test = normalize(test_df, label_encoder, feature_scaler, target_scaler)
    n_encodings = len(np.unique(train[:,0]))

    #Convert format
    train_data, train_target = convert_format(train)
    test_data, test_target = convert_format(test)
    
    return train_data, train_target, test_data, test_target

In [ ]:
def get_loss(trial, quantiles = quantiles):
    #Sample alpha
    alpha = trial.suggest_float("alpha", 1e-5, 0.01)
    loss = trial.suggest_categorical("loss", ["pinball","exp","abs","huber"])
    losses = {
    "pinball": lambda true,pred: pinball_loss(true, pred, tau = quantiles),
    "exp": lambda true,pred: exp_pinball_loss(true, pred, tau = quantiles, alpha = alpha),
    "abs": lambda true,pred: sqrt_pinball_loss(true, pred, tau = quantiles, alpha = alpha),
    "huber": lambda true,pred: huber_pinball_loss(true, pred, tau = quantiles, alpha = alpha)}


    loss_func = losses[loss]
    return loss_func

In [ ]:
def get_optimizer(trial):
    # Copied from optuna tutorial
    kwargs = {}
    optimizer_options = ["Adam", "SGD"]
    optimizer_selected = trial.suggest_categorical("optimizer", optimizer_options)
    if optimizer_selected == "Adam":
        kwargs["learning_rate"] = trial.suggest_float("adam_learning_rate", 1e-5, 1e-2, log=True)
    elif optimizer_selected == "SGD":
        kwargs["learning_rate"] = trial.suggest_float(
            "sgd_opt_learning_rate", 1e-5, 1e-2, log=True
        )
    optimizer = getattr(tf.optimizers, optimizer_selected)(**kwargs)
    return optimizer

In [ ]:
def create_model(trial):
    #Parameters
    dropout_rate = trial.suggest_float("dropout", 0.05, 0.5, log = True)
    embedding_dim = trial.suggest_int("embedding_dim", 2, 6, step = 2)
    n_layers = trial.suggest_int("n_layers",1,2,1)
    n_units_1 = trial.suggest_int("n_units_1",16,128, log =True)
    if n_layers == 2:
        n_units_2 = trial.suggest_int("n_units_2", 16, 128, log = True)
    else:
        n_units_2 = 1
    #Create Model
    class base_model(tf.keras.Model):    
        def __init__(self, n_layers, n_units_1, n_units_2, embedding_dim, dropout_rate, n_embeddings = n_encodings):
            super(base_model, self).__init__()
            #Embedding layers
            self.embedding = Embedding(input_dim = n_embeddings, output_dim = embedding_dim)
            #N_layers
            self.n_layers = n_layers
            #Dropout
            self.dropout = Dropout(dropout_rate)
            #Create Dense layers
            self.hidden = Dense(n_units_1, activation = "relu")
            self.hidden2 = Dense(n_units_2, activation = "relu")
            self.out = Dense(5, activation = "linear")

        def call(self, input_data):
            #Extract data
            features, horizon_emb = input_data
            #Calculate embedding
            emb = self.embedding(horizon_emb)
            emb = tf.squeeze(emb, axis = 1)
            conc = Concatenate(axis = 1)([features, emb])
            #Calculate output
            output = self.hidden(conc)
            if self.n_layers == 2:
                output = self.hidden2(output)
            output = self.dropout(output)
            output = self.out(output)
            return output

    #Train
    model = base_model(n_layers, n_units_1, n_units_2, embedding_dim, dropout_rate)    
    return model

In [ ]:
def learn(model, loss_func, optimizer, batch_size, epochs = 100,  data = data, n_splits = 10):
    #Get data
    data = data.reset_index().drop("index",axis=1)
    fold = KFold(n_splits = n_splits, shuffle = True, random_state = 10)
    split = fold.split(data.index)
    
    #Total loss
    test_loss = 0
    for train_index, test_index in split:
        #Get data
        train_data, train_target, test_data, test_target = get_data(train_index, test_index, data)

        #Compile model
        callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience = 7, min_delta = 1e-5)
        model.compile(optimizer = optimizer, loss = loss_func)
        
        #Normal fit
        history1 = model.fit(x = train_data, y = train_target, validation_split = 0.2, epochs = epochs, batch_size = batch_size, callbacks = [callback], shuffle = True, verbose = False)

        #Calculate loss
        pred = model.predict(test_data)
        total_loss = 0
        for cnt,quantile in enumerate(quantiles):
            loss = mean_pinball_loss(test_target.reshape(-1), pred[:,cnt].reshape(-1), alpha = quantile)
            total_loss += loss

        test_loss += total_loss/len(quantiles)

    return test_loss/n_splits

In [ ]:
def objective(trial):
    # Create Parameters
    batch_size = 2**trial.suggest_int("batch_size",7,10,1)
    epochs = 100
    optimizer = get_optimizer(trial)    
    #Get loss
    loss_func = get_loss(trial)        
    #Get Model
    model = create_model(trial)    
    #Train model
    loss = learn(model = model, loss_func = loss_func, optimizer = optimizer, batch_size = batch_size, epochs = epochs, n_splits = 10)

    return loss

## Run study

In [ ]:
wind_study = opt.create_study(direction = 'minimize')
wind_study.optimize(objective, n_trials = 2)

In [ ]:
print(wind_study.best_value)
wind_study.best_params

# Tune temperature model

## Load data

In [ ]:
data = pd.read_feather("data/berlin_data/historic_data/icon_eps_t_2m.feather")
data.dropna(inplace=True)
#Positional encoding
pos_enc = pd.DataFrame(index=pd.DatetimeIndex(data["obs_tm"]))
pos_enc["Dayofyear"] = pos_enc.index.dayofyear
pos_enc["n_days"] = 365
pos_enc.loc[pos_enc.index.year==2020,"n_days"] = 366
#Calculate actual positional encoding
cos_encoding = np.cos(2*math.pi*pos_enc["Dayofyear"]/pos_enc["n_days"])
data["pos_enc_1"] = cos_encoding.to_numpy()

## Run study

In [ ]:
temp_study = opt.create_study(direction = 'minimize')
temp_study.optimize(objective, n_trials = 2)

In [ ]:
print(temp_study.best_value)
temp_study.best_params

# Compare Loss functions

Compare Loss functions by tuning only loss function for wind model

## Functions

In [ ]:
def normalize(dataframe, label_encoder = None,feature_scaler = None, target_scaler = None, learn = False):
    #Drop unused columns
    data = dataframe.copy()
    data.drop(["init_tm", "met_var", "location",  "ens_var", "obs_tm"], axis = 1, inplace = True)
    data = data.to_numpy()
    if learn == True:
        label_encoder = LabelEncoder()
        feature_scaler = StandardScaler()
        target_scaler = StandardScaler()
        #Learn label encoding for horizons
        label = label_encoder.fit_transform(data[:,0])
        #Learn target scaling
        target_scaled = target_scaler.fit_transform(data[:,1].reshape(-1,1))
        #Learn feature scaling
        feature_scaled = feature_scaler.fit_transform(data[:,2:])
        #Append
        data[:,0] = label
        data[:,1] = target_scaled.reshape(-1)
        data[:,2:] = feature_scaled
        
        return data, label_encoder, feature_scaler, target_scaler
    
    else:
        #Learn labels
        label = label_encoder.transform(data[:,0])
        #Scale target
        target_scaled = target_scaler.transform(data[:,1].reshape(-1,1))
        #Scale features
        feature_scaled = feature_scaler.transform(data[:,2:])
        #Append
        data[:,0] = label
        data[:,1] = target_scaled.reshape(-1)
        data[:,2:] = feature_scaled
        
        return data

In [ ]:
def convert_format(input_data, predict = False):
    #Extract forecast embedding
    horizon_emb = input_data[:,0]
    
    if predict == False:        
        #Extract features
        features = input_data[:,2:]
        # Extract target
        target = np.expand_dims(input_data[:,1],1)
        return [features, horizon_emb], target
    else:
        #Extract features
        features = input_data[:,1:]
        return [features, horizon_emb]

In [ ]:
def train_model(train_data, train_target, validation_data, batch_size, epochs, learning_rate, fine_tuning = True):
    model = base_model()    
    #Define optimizer
    optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)
    #Callbacks
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience = 7, min_delta = 1e-5)
    model.compile(optimizer = optimizer, loss = lambda true,pred: pinball_loss(true, pred, tau = quantiles))
    #model.compile(optimizer = optimizer, loss = lambda true,pred: smooth_pinball_loss(true, pred, tau = quantiles))
    #Normal fit
    history1 = model.fit(x = train_data, y = train_target, validation_data = validation_data, epochs = epochs, batch_size = batch_size, callbacks = [callback], shuffle = True, verbose = False)
    
    #Fine tuning
    if fine_tuning == True:
        enc_horizons = label_encoder.transform(horizons)
        train_filtering = np.isin(train_data[1], enc_horizons)
        train_data_fine = [train_data[0][train_filtering], train_data[1][train_filtering]]
        train_target_fine = train_target[train_filtering]
        #Val filtering
        val_data, val_target = validation_data
        val_filtering = np.isin(val_data[1], enc_horizons)
        val_data_fine = [val_data[0][val_filtering], val_data[1][val_filtering]]
        val_target_fine = val_target[val_filtering]
        validation_data_fine = (val_data_fine, val_target_fine)
        
        #New optimizer
        history2 = model.fit(x = train_data_fine, y = train_target_fine, validation_data = validation_data_fine, epochs = epochs, batch_size = 256, callbacks = [callback], shuffle = True, verbose = False)
    return model, [history1, history2]

## Read data and prepare

In [ ]:
quantiles = [0.025, 0.25, 0.5, 0.75, 0.975]
horizons = [36, 48 ,60, 72, 84]
n_encodings = 65

In [ ]:
#Wind data
wind_data = pd.read_feather("data/berlin_data/historic_data/icon_eps_wind_10m.feather")
#Pressure data
pressure_data = pd.read_feather("data/berlin_data/historic_data/icon_eps_mslp.feather")
pressure_data.rename({"ens_mean":"mean_pressure"}, axis = 1, inplace = True)
#Cloud data
cloud_data = pd.read_feather("data/berlin_data/historic_data/icon_eps_clct.feather")
cloud_data.rename({"ens_mean":"cloud_coverage"}, axis = 1, inplace = True)
#Vmax data
max_data = pd.read_feather("data/berlin_data/historic_data/icon_eps_vmax_10m.feather")
max_data.rename({"ens_mean":"vmax"}, axis = 1, inplace = True)


data = wind_data.merge(pressure_data[["init_tm","fcst_hour","mean_pressure"]], on = ["init_tm","fcst_hour"], how = "left")
data = data.merge(cloud_data[["init_tm","fcst_hour","cloud_coverage"]], on = ["init_tm","fcst_hour"], how = "left")
data = data.merge(max_data[["init_tm","fcst_hour","vmax"]], on = ["init_tm","fcst_hour"], how = "left")
#Replace vmax NaNs by mean
vmax_mean = data["vmax"].mean()
data.loc[:,"vmax"].fillna(vmax_mean, inplace = True)
data.dropna(inplace=True)

#Positional encoding
pos_enc = pd.DataFrame(index=pd.DatetimeIndex(data["obs_tm"]))
pos_enc["Dayofyear"] = pos_enc.index.dayofyear
pos_enc["n_days"] = 365
pos_enc.loc[pos_enc.index.year==2020,"n_days"] = 366
#Calculate actual positional encoding
cos_encoding = np.cos(2*math.pi*pos_enc["Dayofyear"]/pos_enc["n_days"])
data["pos_enc_1"] = cos_encoding.to_numpy()

## Create study object

In [ ]:
def get_data(train_index, test_index, data):
    #Get split
    train_df = data.loc[train_index]
    test_df = data.loc[test_index]

    #Normalize data
    train, label_encoder, feature_scaler, target_scaler = normalize(train_df, learn = True)
    test = normalize(test_df, label_encoder, feature_scaler, target_scaler)
    n_encodings = len(np.unique(train[:,0]))

    #Convert format
    train_data, train_target = convert_format(train)
    test_data, test_target = convert_format(test)
    
    return train_data, train_target, test_data, test_target

In [ ]:
def get_loss(loss, alpha, quantiles = quantiles):
    #Sample alpha
    losses = {
    "pinball": lambda true,pred: pinball_loss(true, pred, tau = quantiles),
    "exp": lambda true,pred: exp_pinball_loss(true, pred, tau = quantiles, alpha = alpha),
    "abs": lambda true,pred: sqrt_pinball_loss(true, pred, tau = quantiles, alpha = alpha),
    "huber": lambda true,pred: huber_pinball_loss(true, pred, tau = quantiles, alpha = alpha)
    }
    loss_func = losses[loss]
    return loss_func

In [ ]:
def create_model():
    #Parameters
    dropout_rate = 0.1
    n_layers = 2
    n_units_1 = 58
    if n_layers == 2:
        n_units_2 = 64
    else:
        n_units_2 = 1
    #Create Model
    class base_model(tf.keras.Model):    
        def __init__(self, n_layers, n_units_1, n_units_2, dropout_rate, n_embeddings = n_encodings):
            super(base_model, self).__init__()
            #Embedding layers
            self.embedding = Embedding(input_dim = n_embeddings, output_dim = 4)
            #N_layers
            self.n_layers = n_layers
            #Dropout
            self.dropout = Dropout(dropout_rate)
            #Create Dense layers
            self.hidden = Dense(n_units_1, activation = "relu")
            self.hidden2 = Dense(n_units_2, activation = "relu")
            self.out = Dense(5, activation = "linear")

        def call(self, input_data):
            #Extract data
            features, horizon_emb = input_data
            #Calculate embedding
            emb = self.embedding(horizon_emb)
            emb = tf.squeeze(emb, axis = 1)
            conc = Concatenate(axis = 1)([features, emb])
            #Calculate output
            output = self.hidden(conc)
            if self.n_layers == 2:
                output = self.hidden2(output)
            output = self.dropout(output)
            output = self.out(output)
            return output

    #Train
    model = base_model(n_layers, n_units_1, n_units_2, dropout_rate)    
    return model

In [ ]:
def learn(model, loss_func, optimizer, batch_size, epochs = 100,  data = data, n_splits = 10):
    #Get data
    data = data.reset_index().drop("index",axis=1)
    fold = KFold(n_splits = n_splits, shuffle = True, random_state = 10)
    split = fold.split(data.index)
    
    #Total loss
    test_loss = 0
    for train_index, test_index in split:
        #Get data
        train_data, train_target, test_data, test_target = get_data(train_index, test_index, data)

        #Compile model
        callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience = 7, min_delta = 1e-5)
        model.compile(optimizer = optimizer, loss = loss_func)
        
        #Normal fit
        history1 = model.fit(x = train_data, y = train_target, validation_split = 0.2, epochs = epochs, batch_size = batch_size, callbacks = [callback], shuffle = True, verbose = False)

        #Calculate loss
        pred = model.predict(test_data)
        total_loss = 0
        for cnt,quantile in enumerate(quantiles):
            loss = mean_pinball_loss(test_target.reshape(-1), pred[:,cnt].reshape(-1), alpha = quantile)
            total_loss += loss

        test_loss += total_loss/len(quantiles)

    return test_loss/n_splits

## Run experiment

In [ ]:
#Define parameters
alpha_space = np.logspace(start = np.log10(1e-5), stop = np.log10(1), num = 30)
batch_size = 128
epochs = 100
optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001)

In [ ]:
# Create evaluation dataframe
results = pd.DataFrame(index = np.arange(120), columns = ["Method", "alpha", "Pinball loss"])

for cnt_loss, loss in enumerate(["pinball","exp","abs","huber"]):
    for cnt_alpha,alpha in enumerate(alpha_space):
        loss_func = get_loss(loss,alpha)    
        model = create_model() 
        pinball_loss = learn(model = model, loss_func = loss_func, optimizer = optimizer, batch_size = batch_size, epochs = epochs, n_splits = 5)
        results.loc[cnt_loss*30+cnt_alpha] = [loss, alpha, pinball_loss]

In [ ]:
results.to_pickle("loss_experiment_results.pickle")